## News Scraper

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import string
from textblob import TextBlob
import plotly.express as px
import pandas as pd
import csv
import json
import time

In [17]:
start_time = time.time()
option = webdriver.ChromeOptions()
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--disable-dev-sh-usage')
# Replace the path with your chromedriver location
driver = webdriver.Chrome('PATH')
 
driver.get('https://www.aljazeera.com/where/mozambique/')
soup = BeautifulSoup(driver.page_source, 'html.parser') # Parsing content using beautifulsoup. Notice driver.page_source instead of page.content
 
urls = soup.select(".gc--type-post .u-clickable-card__link")
totalScrapedInfo = [] 
for url in urls:
    driver.get('https://www.aljazeera.com' + url['href'])
    title = driver.find_elements(by=By.CSS_SELECTOR, value=".article-header h1")
    if not title: continue
    contents = driver.find_elements(by=By.CSS_SELECTOR, value=".wysiwyg p")
    date = driver.find_elements(by=By.CSS_SELECTOR, value=".date-simple span")
    content = [c.text for c in contents if c!=' ']
    content = " ".join(content)
    
    scrapedInfo = {
        "title": title[0].text,
        "content": content,
        "time": date[1].text
    }
    totalScrapedInfo.append(scrapedInfo)
    if len(totalScrapedInfo)==10:
        break
    

/Users/luerlyu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [10]:
file = open('news.json', mode='w', encoding='utf-8')
file.write(json.dumps(totalScrapedInfo))
 
writer = csv.writer(open("news.csv", 'w'))
for movie in totalScrapedInfo:
    writer.writerow(movie.values())

In [11]:
news = pd.read_csv("news.csv", names = ['title','content','time'])

In [12]:
news

,title,content,time
0,Floods hit South Africa’s KwaZulu-Natal provin...,Heavy rains in South Africa have forced hundre...,23 May 2022
1,Mozambique: Cyclone Gombe death toll rises to 53,Tropical Cyclone Gombe has killed at least 53 ...,18 Mar 2022
2,Mozambique announces new prime minister after ...,Mozambique President Filipe Nyusi has appointe...,4 Mar 2022
3,Analysis: Can African gas replace Russian supp...,"Lagos, Nigeria – On February 22, German Chance...",1 Mar 2022
4,Dozens dead from Tropical Storm Ana in souther...,The death toll from a storm that struck three ...,27 Jan 2022
5,Southern Africa bloc SADC extends Mozambique m...,Southern African regional bloc SADC at a summi...,12 Jan 2022
6,"In Mozambique, Kagame says Rwandan troops’ wor...",Rwandan President Paul Kagame has arrived in M...,24 Sep 2021
7,"Rwanda, Mozambique forces recapture port city ...",Rwandan forces deployed last month to help Moz...,8 Aug 2021
8,"Rwanda deploys 1,000 soldiers to Mozambique’s ...","Rwanda has started deploying 1,000-strong forc...",10 Jul 2021
9,Southern African nations agree to deploy force...,The Southern African Development Community (SA...,23 Jun 2021


## pre-process

the scraped text data are clean already(without images and dates etc), so in this step some nlp pre-process cleaning is performed: remove punctuations and lower the texts.

In [13]:
news["text"] = news["title"]+" "+news["content"]

In [14]:
def clean(text):
    text = text.lower()
    text = text.split()
    text = [t for t in text if t not in string.punctuation]
    return " ".join(text)
news["text"] = news["text"].apply(clean)

## sentiment analysis

In [ ]:
news['sentiment'] = news['text'].apply(lambda x: TextBlob(x).sentiment)
news['sentiment'] = news['sentiment'].apply(lambda x: x[0])

In [18]:
print("--- operation time: %s seconds ---" % (time.time() - start_time))

--- operation time: 20.32680106163025 seconds ---


## plot

In [ ]:
fig = px.line( news, x = 'time' ,
              y = 'sentiment',
              title = 'sentiment changes for the 10 most recent news')
fig.show()